In [ ]:
### REMOVE CELL
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession



# Group similar features
trasimitted_in_features = ["IN_BYTES", "IN_PKTS"]
trasmitted_out_features = ["OUT_BYTES", "OUT_PKTS"]
retransimtted_in_features = ["RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS"]
retransimtted_out_features = ["RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS"]
throughput_in_features = ["SRC_TO_DST_SECOND_BYTES", "SRC_TO_DST_AVG_THROUGHPUT"]
throughput_out_features = ["DST_TO_SRC_AVG_THROUGHPUT", "DST_TO_SRC_SECOND_BYTES"]
ttl_features = ["MIN_TTL", "MAX_TTL"]
tcp_flags_features = ["CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "TCP_FLAGS"]
icmp_features = ["ICMP_TYPE", "ICMP_IPV4_TYPE"]
flow_max_features = ["LONGEST_FLOW_PKT", "MAX_IP_PKT_LEN"]
flow_min_features = ["SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN"]
flow_duration_features = ["FLOW_DURATION_MILLISECONDS"]
tcp_in_features = ["TCP_WIN_MAX_IN"]
tcp_out_features = ["TCP_WIN_MAX_OUT"]

# Assemble and scale each group
assemblers_and_scalers = []

def create_assembler_and_scaler(inputCols, outputCol):
    assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)
    scaler = StandardScaler(inputCol=outputCol, outputCol=f"scaled_{outputCol}", withStd=True, withMean=True)
    return assembler, scaler

grouped_features = {
    "trasmitted_in": trasimitted_in_features,
    "trasmitted_out": trasmitted_out_features,
    "retransimtted_in": retransimtted_in_features,
    "retransimtted_out": retransimtted_out_features,
    "throughput_in": throughput_in_features,
    "throughput_out": throughput_out_features,
    "ttl": ttl_features,
    "tcp_flags": tcp_flags_features,
    "icmp": icmp_features,
    "flow_max": flow_max_features,
    "flow_min": flow_min_features,
    "flow_duration": flow_duration_features,
    "tcp_in": tcp_in_features,
    "tcp_out": tcp_out_features,
}

for group_name, features in grouped_features.items():
    assembler, scaler = create_assembler_and_scaler(features, f"{group_name}_features")
    assemblers_and_scalers.append(assembler)
    assemblers_and_scalers.append(scaler)

# Assemble all scaled features into a single feature vector
all_scaled_features = [f"scaled_{group_name}_features" for group_name in grouped_features.keys()]
final_assembler = VectorAssembler(inputCols=all_scaled_features, outputCol="final_features")

# Add PCA
COMPONENTS = 3  # Number of desired principal components
pca = PCA(k=COMPONENTS, inputCol="final_features", outputCol="pca_features")

# Create the complete pipeline
pipeline = Pipeline(stages=assemblers_and_scalers + [final_assembler, pca])

# Fit the pipeline and transform the data
pca_model = pipeline.fit(encoded_train_data)
pca_result = pca_model.transform(encoded_train_data)





In [ ]:
## REMOVE CELL

from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
## COPY THIS AND TRY WITH ML DATASET
FEATURES = all_scaled_features
COMPONENTS = 3
def PCA_analysis_variance(data, features=FEATURES, components=COMPONENTS):
    # Esegui l'analisi PCA e ottieni i risultati
    assembler = VectorAssembler(inputCols=features, outputCol="features")
    scaler = StandardScaler(
        inputCol="features", outputCol="scaled_features", withStd=True, withMean=True
    )
    pca = PCA(k=components, inputCol="scaled_features", outputCol="pca_features")
    pipeline = Pipeline(stages=[assembler, scaler, pca])
    pca_model = pipeline.fit(data)
    pca_result = pca_model.transform(data)

    return pca_result, pca_model


# Perform PCA analysis
pca_result, pca_model = PCA_analysis_variance(encoded_train_data)

IllegalArgumentException: scaled_trasmitted_in_features does not exist. Available: L4_SRC_PORT, L4_DST_PORT, PROTOCOL, L7_PROTO, IN_BYTES, IN_PKTS, OUT_BYTES, OUT_PKTS, TCP_FLAGS, CLIENT_TCP_FLAGS, SERVER_TCP_FLAGS, FLOW_DURATION_MILLISECONDS, DURATION_IN, DURATION_OUT, MIN_TTL, MAX_TTL, LONGEST_FLOW_PKT, SHORTEST_FLOW_PKT, MIN_IP_PKT_LEN, MAX_IP_PKT_LEN, SRC_TO_DST_SECOND_BYTES, DST_TO_SRC_SECOND_BYTES, RETRANSMITTED_IN_BYTES, RETRANSMITTED_IN_PKTS, RETRANSMITTED_OUT_BYTES, RETRANSMITTED_OUT_PKTS, SRC_TO_DST_AVG_THROUGHPUT, DST_TO_SRC_AVG_THROUGHPUT, NUM_PKTS_UP_TO_128_BYTES, NUM_PKTS_128_TO_256_BYTES, NUM_PKTS_256_TO_512_BYTES, NUM_PKTS_512_TO_1024_BYTES, NUM_PKTS_1024_TO_1514_BYTES, TCP_WIN_MAX_IN, TCP_WIN_MAX_OUT, ICMP_TYPE, ICMP_IPV4_TYPE, DNS_QUERY_ID, DNS_QUERY_TYPE, DNS_TTL_ANSWER, FTP_COMMAND_RET_CODE, attack_index, dst_subnet_index, src_subnet_index